In [2]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.8 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 14.7 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=be6056412a2be9c076de8cc08ee68513c85550e48f327f91893c99f0489a6f27
  Stored in directory: /root/.cache/pip/wheels/5a/54/9b/a89cac960efb57c4c35d41cc7c9f7b80daa21108bc376339b7
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pandasql

Note: you may need to restart the kernel to use updated packages.


### In questo notebook lavoreremo con dei datasets contenenti dati fittizi sulla vendita di chitarre. Verranno proposte alcune query in SQL e le relative controparti in PySpark.

In [70]:
# Librerie
import pyspark.sql.functions as F
import pyspark.sql.types as T
import pandas as pd
from pyspark.sql import SparkSession
from pandasql import sqldf
from pyspark.sql.window import Window

In [5]:
# Creazione SparkSession
spark = (SparkSession.builder.appName("Guitars").getOrCreate())

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/19 19:25:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
# Path dei csv files di nostro interesse
path_prodotti = "/kaggle/input/sql-pyspark-guitars/Prodotti.csv"
path_vendite = "/kaggle/input/sql-pyspark-guitars/Vendite.csv"
path_venditori = "/kaggle/input/sql-pyspark-guitars/Venditori.csv"

In [7]:
# Creazione dei df per Spark

prodotti_spark = (spark.read.format("csv")
              .option("header", True)
              .option("delimiter", ";")
              .load(path_prodotti))

vendite_spark = (spark.read.format("csv")
              .option("header", True)
              .option("delimiter", ";")
              .load(path_vendite))

venditori_spark = (spark.read.format("csv")
              .option("header", True)
              .option("delimiter", ";")
              .load(path_venditori))

In [8]:
prodotti_spark.show(5)

+---+------+--------+------+---------+
| ID| Marca| Modello|Prezzo|Categoria|
+---+------+--------+------+---------+
|  1|Gibson|Les Paul|  2300| Chitarre|
|  2|Gibson|      SG|  1700| Chitarre|
|  3|Gibson|Flying V|  2200| Chitarre|
|  4|Gibson|Explorer|  2100| Chitarre|
|  5|Gibson|    J-45|  1900| Chitarre|
+---+------+--------+------+---------+
only showing top 5 rows



In [9]:
vendite_spark.show(5)

+----------+----+---------+--------+--------+
|      Data|Anno|Venditore|Prodotto|Quantita|
+----------+----+---------+--------+--------+
|11/01/2022|2022|        2|       1|       1|
|21/02/2022|2022|        3|       6|       2|
|24/03/2022|2022|        1|      11|       2|
|02/04/2022|2022|        4|       2|       4|
|12/04/2022|2022|        8|       3|       2|
+----------+----+---------+--------+--------+
only showing top 5 rows



In [10]:
venditori_spark.show(5)

+---+------+--------+---+
| ID|  Nome| Cognome|Eta|
+---+------+--------+---+
|  1|Andrea|  Assini| 25|
|  2| Bruno| Bettini| 22|
|  3| Carlo|Cicerini| 21|
|  4| Dario|  Desini| 29|
|  5| Elena| Ettorre| 34|
+---+------+--------+---+
only showing top 5 rows



In [11]:
# Creazione dei df per SQL

prodotti_sql = pd.read_csv(path_prodotti, delimiter=";")
vendite_sql = pd.read_csv(path_vendite, delimiter=";")
venditori_sql = pd.read_csv(path_venditori, delimiter=";")

In [12]:
prodotti_sql.head(5)

,ID,Marca,Modello,Prezzo,Categoria
0,1,Gibson,Les Paul,2300,Chitarre
1,2,Gibson,SG,1700,Chitarre
2,3,Gibson,Flying V,2200,Chitarre
3,4,Gibson,Explorer,2100,Chitarre
4,5,Gibson,J-45,1900,Chitarre


In [13]:
vendite_sql.head(5)

,Data,Anno,Venditore,Prodotto,Quantita
0,11/01/2022,2022,2,1,1
1,21/02/2022,2022,3,6,2
2,24/03/2022,2022,1,11,2
3,02/04/2022,2022,4,2,4
4,12/04/2022,2022,8,3,2


In [14]:
venditori_sql.head(5)

,ID,Nome,Cognome,Eta
0,1,Andrea,Assini,25
1,2,Bruno,Bettini,22
2,3,Carlo,Cicerini,21
3,4,Dario,Desini,29
4,5,Elena,Ettorre,34


### 1) Selezionare i prodotti di marca Gibson

In [15]:
# SQL
query = """SELECT * FROM prodotti_sql 
        WHERE Marca = 'Gibson'"""
result = sqldf(query)
result

,ID,Marca,Modello,Prezzo,Categoria
0,1,Gibson,Les Paul,2300,Chitarre
1,2,Gibson,SG,1700,Chitarre
2,3,Gibson,Flying V,2200,Chitarre
3,4,Gibson,Explorer,2100,Chitarre
4,5,Gibson,J-45,1900,Chitarre


In [16]:
# PySpark
result = prodotti_spark.filter(F.col("Marca") == "Gibson")
result.show()

+---+------+--------+------+---------+
| ID| Marca| Modello|Prezzo|Categoria|
+---+------+--------+------+---------+
|  1|Gibson|Les Paul|  2300| Chitarre|
|  2|Gibson|      SG|  1700| Chitarre|
|  3|Gibson|Flying V|  2200| Chitarre|
|  4|Gibson|Explorer|  2100| Chitarre|
|  5|Gibson|    J-45|  1900| Chitarre|
+---+------+--------+------+---------+



### 2) Selezionare i prodotti di marca "Fender" con un prezzo superiore a 2000 euro

In [17]:
# SQL
query = """SELECT * FROM prodotti_sql 
        WHERE Marca = 'Fender' and Prezzo > 2000"""
result = sqldf(query)
result

,ID,Marca,Modello,Prezzo,Categoria
0,8,Fender,Jaguar,2100,Chitarre
1,12,Fender,Precision,2500,Bassi
2,13,Fender,Jazz,2800,Bassi


In [18]:
# PySpark
result = prodotti_spark.filter((F.col("Marca")=="Fender") & (F.col("Prezzo")>2000))
result.show()

+---+------+---------+------+---------+
| ID| Marca|  Modello|Prezzo|Categoria|
+---+------+---------+------+---------+
|  8|Fender|   Jaguar|  2100| Chitarre|
| 12|Fender|Precision|  2500|    Bassi|
| 13|Fender|     Jazz|  2800|    Bassi|
+---+------+---------+------+---------+



### 3) Restituire il numero totale di vendite anno per anno

In [19]:
# SQL
query = """SELECT Anno, count(*) AS Vendite_Anno 
        FROM vendite_sql GROUP BY Anno"""
result = sqldf(query)
result

,Anno,Vendite_Anno
0,2022,15
1,2023,15


In [20]:
# PySpark
result = vendite_spark.groupby(F.col("Anno")).agg(F.count('*').alias("Vendite_Anno"))
result.show()

+----+------------+
|Anno|Vendite_Anno|
+----+------------+
|2022|          15|
|2023|          15|
+----+------------+



### 4) Restituire il numero totale di vendite per marca

In [21]:
# SQL
query = """SELECT prodotti_sql.Marca, count(*) AS Vendite_Marca 
        FROM vendite_sql INNER JOIN prodotti_sql 
        ON vendite_sql.Prodotto = prodotti_sql.ID 
        GROUP BY prodotti_sql.Marca"""
result = sqldf(query)
result

,Marca,Vendite_Marca
0,Fender,12
1,Gibson,13
2,Ibanez,5


In [22]:
# PySpark
result = vendite_spark \
        .join(prodotti_spark, vendite_spark["Prodotto"]==prodotti_spark["ID"]) \
        .groupby(prodotti_spark.Marca) \
        .agg(F.count("*")) \
        .orderBy(prodotti_spark.Marca)
result.show()

+------+--------+
| Marca|count(1)|
+------+--------+
|Fender|      12|
|Gibson|      13|
|Ibanez|       5|
+------+--------+



### 4) Restituire il numero totale di vendite anno per anno, per marca

In [23]:
# SQL
query = """SELECT vendite_sql.Anno, prodotti_sql.Marca, count(*) AS Vendite_Marca 
        FROM vendite_sql INNER JOIN prodotti_sql 
        ON vendite_sql.Prodotto = prodotti_sql.ID 
        GROUP BY vendite_sql.Anno, prodotti_sql.Marca"""
result = sqldf(query)
result

,Anno,Marca,Vendite_Marca
0,2022,Fender,7
1,2022,Gibson,7
2,2022,Ibanez,1
3,2023,Fender,5
4,2023,Gibson,6
5,2023,Ibanez,4


In [24]:
# PySpark
result = vendite_spark \
        .join(prodotti_spark, vendite_spark["Prodotto"]==prodotti_spark["ID"]) \
        .groupby(vendite_spark.Anno, prodotti_spark.Marca) \
        .agg(F.count("*")) \
        .orderBy(vendite_spark.Anno, prodotti_spark.Marca)
result.show()

+----+------+--------+
|Anno| Marca|count(1)|
+----+------+--------+
|2022|Fender|       7|
|2022|Gibson|       7|
|2022|Ibanez|       1|
|2023|Fender|       5|
|2023|Gibson|       6|
|2023|Ibanez|       4|
+----+------+--------+



### 5) Calcolare il ricavo di ciascuna vendita

In [26]:
# SQL
query = """SELECT vendite_sql.Data, vendite_sql.Quantita*prodotti_sql.Prezzo AS Ricavo
        FROM vendite_sql INNER JOIN prodotti_sql 
        ON vendite_sql.Prodotto = prodotti_sql.ID""" 
result = sqldf(query)
result

,Data,Ricavo
0,11/01/2022,2300
1,21/02/2022,3600
2,24/03/2022,3600
3,02/04/2022,6800
4,12/04/2022,4400
5,17/05/2022,2100
6,21/05/2022,2100
7,11/06/2022,9200
8,12/07/2022,3800
9,15/08/2022,2100


In [27]:
# PySpark 
result = vendite_spark\
        .join(prodotti_spark, vendite_spark["Prodotto"]==prodotti_spark["ID"])\
        .withColumn("Ricavo", F.col("Quantita")*F.col("Prezzo"))\
        .select("Data", "Ricavo")

result.show()

+----------+------+
|      Data|Ricavo|
+----------+------+
|11/01/2022|2300.0|
|21/02/2022|3600.0|
|24/03/2022|3600.0|
|02/04/2022|6800.0|
|12/04/2022|4400.0|
|17/05/2022|2100.0|
|21/05/2022|2100.0|
|11/06/2022|9200.0|
|12/07/2022|3800.0|
|15/08/2022|2100.0|
|20/09/2022|3800.0|
|05/10/2022|4200.0|
|27/11/2022|8500.0|
|29/11/2022|1900.0|
|04/12/2022|2100.0|
|11/01/2023|6000.0|
|21/02/2023|4200.0|
|24/03/2023|3600.0|
|02/04/2023|6600.0|
|12/04/2023|5100.0|
+----------+------+
only showing top 20 rows



### 6) Ricavo totale per venditore

In [37]:
# SQL
query = """SELECT venditori_sql.Nome, venditori_sql.Cognome, SUM(vendite_sql.Quantita*prodotti_sql.Prezzo) AS Ricavo_Venditore
        FROM vendite_sql 
        INNER JOIN prodotti_sql 
        ON vendite_sql.Prodotto = prodotti_sql.ID
        INNER JOIN venditori_sql
        ON vendite_sql.Venditore = venditori_sql.ID
        GROUP BY venditori_sql.Nome, venditori_sql.Cognome
        ORDER BY Ricavo_Venditore DESC""" 
result = sqldf(query)
result

,Nome,Cognome,Ricavo_Venditore
0,Carlo,Cicerini,17900
1,Bruno,Bettini,17800
2,Dario,Desini,17800
3,Elena,Ettorre,17400
4,Ivana,Itteri,17400
5,Giulia,Giorgini,12000
6,Andrea,Assini,11400
7,Federica,Fierini,5800


In [34]:
# PySpark 
result = vendite_spark\
        .join(venditori_spark, vendite_spark["Venditore"] == venditori_spark["ID"])\
        .join(prodotti_spark, vendite_spark["Prodotto"] == prodotti_spark["ID"])\
        .withColumn("Ricavo", F.col("Quantita")*F.col("Prezzo"))\
        .groupBy("Venditore", "Nome", "Cognome")\
        .agg(F.sum("Ricavo").alias("Ricavo_Venditore"))\
        .select("Nome", "Cognome", "Ricavo_Venditore")\
        .orderBy(F.col("Ricavo_Venditore").desc())

result.show()

+--------+--------+----------------+
|    Nome| Cognome|Ricavo_Venditore|
+--------+--------+----------------+
|   Carlo|Cicerini|         17900.0|
|   Dario|  Desini|         17800.0|
|   Bruno| Bettini|         17800.0|
|   Elena| Ettorre|         17400.0|
|   Ivana|  Itteri|         17400.0|
|  Giulia|Giorgini|         12000.0|
|  Andrea|  Assini|         11400.0|
|Federica| Fierini|          5800.0|
+--------+--------+----------------+



### 7) Numero di prodotti diversi venduti anno per anno

In [44]:
# SQL
query = """SELECT vendite_sql.Anno, COUNT(DISTINCT prodotti_sql.Modello)
        FROM vendite_sql 
        INNER JOIN prodotti_sql ON vendite_sql.Prodotto = prodotti_sql.ID
        INNER JOIN venditori_sql ON vendite_sql.Venditore = venditori_sql.ID
        GROUP BY vendite_sql.Anno
        ORDER BY vendite_sql.Anno""" 

result = sqldf(query)
result

,Anno,COUNT(DISTINCT prodotti_sql.Modello)
0,2022,9
1,2023,11


In [47]:
# PySpark
result = vendite_spark\
        .join(venditori_spark, vendite_spark["Venditore"] == venditori_spark["ID"])\
        .join(prodotti_spark, vendite_spark["Prodotto"] == prodotti_spark["ID"])\
        .groupBy("Anno")\
        .agg(F.countDistinct("Modello"))\
        .orderBy("Anno")

result.show()

+----+--------------+
|Anno|count(Modello)|
+----+--------------+
|2022|             9|
|2023|            11|
+----+--------------+



### 8) Restituire la prima vendita per ogni venditore

In [50]:
# SQL
query = """SELECT venditori_sql.Nome, venditori_sql.Cognome, vendite_sql.Data
        FROM vendite_sql INNER JOIN venditori_sql ON vendite_sql.Venditore = venditori_sql.ID
        WHERE vendite_sql.Data = (
            SELECT min(v2.Data)
            FROM vendite_sql v2
            WHERE v2.Venditore = vendite_sql.Venditore
        )
        ORDER BY venditori_sql.Nome, venditori_sql.Cognome""" 

result = sqldf(query)
result

,Nome,Cognome,Data
0,Andrea,Assini,15/08/2022
1,Bruno,Bettini,02/04/2023
2,Carlo,Cicerini,04/12/2023
3,Dario,Desini,02/04/2022
4,Elena,Ettorre,05/10/2022
5,Federica,Fierini,17/05/2022
6,Giulia,Giorgini,12/07/2023
7,Ivana,Itteri,05/10/2023


In [78]:
# PySpark

result = vendite_spark \
        .join(venditori_spark, vendite_spark["Venditore"] == venditori_spark["ID"])\
        .withColumn("min_data", F.min("Data").over(Window.partitionBy("Venditore").orderBy("Data")))\
        .where(F.col("Data") == F.col("min_data"))\
        .select("Nome", "Cognome", "Data")\
        .orderBy("Nome", "Cognome")

result.show()

+--------+--------+----------+
|    Nome| Cognome|      Data|
+--------+--------+----------+
|  Andrea|  Assini|15/08/2022|
|   Bruno| Bettini|02/04/2023|
|   Carlo|Cicerini|04/12/2023|
|   Dario|  Desini|02/04/2022|
|   Elena| Ettorre|05/10/2022|
|Federica| Fierini|17/05/2022|
|  Giulia|Giorgini|12/07/2023|
|   Ivana|  Itteri|05/10/2023|
+--------+--------+----------+

